# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, regexp_replace, col, substring
from select import select
import configparser
import os

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [2]:
spark = SparkSession.builder.getOrCreate()

# # I94 Immigration Data pull
# spark = SparkSession.builder.\
# config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
# .enableHiveSupport().getOrCreate()

22/09/03 14:17:43 WARN Utils: Your hostname, Samars-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.29.188 instead (on interface en0)
22/09/03 14:17:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/03 14:17:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
output_data='output_data/'

In [ ]:
# reading US Immigration data
immigration_data = spark.read.parquet("sas_data")
print(immigration_data.count())
immigration_data.limit(10).toPandas()


In [ ]:
immigration_data.printSchema()

In [ ]:
# reading US Demographics data

demographic_data = pd.read_csv('us-cities-demographics.csv', sep=';')
print(demographic_data.info())

In [ ]:
demographic_data

In [ ]:
# reading US airport codes data
airport_code_data = pd.read_csv('airport-codes_csv.csv')
print(airport_code_data.info())
airport_code_data

In [4]:
# reading Global temperature data
global_temperature_data = pd.read_csv('temp_data/GlobalLandTemperaturesByCity.csv')

# fname = ('../../data2/GlobalLandTemperaturesByCity.csv')
# global_temperature_data = pd.read_csv(fname)

print(global_temperature_data.info())
global_temperature_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   dt                             object 
 1   AverageTemperature             float64
 2   AverageTemperatureUncertainty  float64
 3   City                           object 
 4   Country                        object 
 5   Latitude                       object 
 6   Longitude                      object 
dtypes: float64(2), object(5)
memory usage: 459.2+ MB
None


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
...,...,...,...,...,...,...,...
8599207,2013-05-01,11.464,0.236,Zwolle,Netherlands,52.24N,5.26E
8599208,2013-06-01,15.043,0.261,Zwolle,Netherlands,52.24N,5.26E
8599209,2013-07-01,18.775,0.193,Zwolle,Netherlands,52.24N,5.26E
8599210,2013-08-01,18.025,0.298,Zwolle,Netherlands,52.24N,5.26E


In [5]:
df=spark.createDataFrame(global_temperature_data)
df.write.mode("overwrite").parquet(os.path.join(output_data, 'global_temperature_data'), partitionBy=['dt', 'Country'])

22/09/03 14:27:25 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 328653 ms exceeds timeout 120000 ms
22/09/03 14:27:25 WARN SparkContext: Killing executors is not supported by current scheduler.
22/09/03 14:30:29 WARN TaskSetManager: Stage 0 contains a task of very large size (30737 KiB). The maximum recommended task size is 1000 KiB.


Exception in thread "refresh progress" java.lang.OutOfMemoryError: Java heap space
Exception in thread "Spark Context Cleaner" java.lang.OutOfMemoryError: Java heap space


22/09/03 14:30:32 ERROR Utils: uncaught error in thread Spark Context Cleaner, stopping SparkContext
java.lang.OutOfMemoryError: Java heap space
22/09/03 14:30:32 ERROR Executor: Exception in task 5.0 in stage 0.0 (TID 5)
java.lang.OutOfMemoryError: Java heap space
22/09/03 14:30:32 ERROR Executor: Exception in task 1.0 in stage 0.0 (TID 1)
java.lang.OutOfMemoryError: Java heap space
22/09/03 14:30:32 ERROR Executor: Exception in task 8.0 in stage 0.0 (TID 8)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newArray(Native Method)
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2145)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1745)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2181)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1745)
	at java.base/java.io.ObjectInputStream$Field

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/utils.py", line 190, in deco
    return f(*a, **kw)
  File "/opt/anaconda3/lib/python3.9/site-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/anaconda3/lib/python3

Py4JError: py4j.reflection does not exist in the JVM

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
df = spark.read.text('I94_SAS_Labels_Descriptions.SAS')
df.show(df.count(), False)

In [ ]:
i94country = df.filter("value like '% =  %'").withColumn('value', regexp_replace(col("value"), " =  ", "="))\
        .withColumn('value', regexp_replace(col("value"), "'", ""))\
        .withColumn('value', regexp_replace(col("value"), "   ", ""))\
        .withColumn('value', regexp_replace(col("value"), " ;", ""))\
        .selectExpr('substring(value, 0, 3) as code', 'substring(value, 5, length(value)) as country')
i94country.show(i94country.count(), False)
i94country.count()

In [ ]:
i94country.write.mode("overwrite").parquet(os.path.join(output_data, 'i94country'))

In [ ]:
i94port = df.filter("value like '%\t=\t%'").withColumn('value', regexp_replace(col("value"), "\t=\t", "="))\
        .withColumn('value', regexp_replace(col("value"), "'", ""))\
        .withColumn('value', regexp_replace(col("value"), "   ", ""))\
        .withColumn('value', regexp_replace(col("value"), " =", "="))\
        .selectExpr('substring(value, 0, 3) as code', 'substring(value, 5, length(value)) as port')
i94port.show(i94port.count(), False)
i94port.count()

In [ ]:
i94port.write.mode("overwrite").parquet(os.path.join(output_data, 'i94port'))

In [ ]:
i94mode = df.filter("value like '\t% = %'").withColumn('value', regexp_replace(col("value"), " = ", "="))\
        .withColumn('value', regexp_replace(col("value"), "\t", ""))\
        .withColumn('value', regexp_replace(col("value"), "'", ""))\
        .withColumn('value', regexp_replace(col("value"), " ;", ""))\
        .selectExpr('substring(value, 0, 1) as code', 'substring(value, 3, length(value)) as mode')
i94mode.show(i94port.count(), False)
i94mode.count()

In [ ]:
i94mode.write.mode("overwrite").parquet(os.path.join(output_data, 'i94mode'))

In [ ]:

i94address_state = df.filter("value like '\t%\\'=\\'%'").withColumn('value', regexp_replace(col("value"), " = ", "="))\
        .withColumn('value', regexp_replace(col("value"), "\t", ""))\
        .withColumn('value', regexp_replace(col("value"), "'", ""))\
        .withColumn('value', regexp_replace(col("value"), " ;", ""))\
        .selectExpr('substring(value, 0, 2) as code', 'substring(value, 4, length(value)) as state')
i94address_state.show(i94address_state.count(), False)
i94address_state.count()

In [ ]:
i94address_state.write.mode("overwrite").parquet(os.path.join(output_data, 'i94address_state'))

In [ ]:
i94visa = df.filter("value like '   _ =%'").withColumn('value', regexp_replace(col("value"), " = ", "="))\
        .withColumn('value', regexp_replace(col("value"), "   ", ""))\
        .selectExpr('substring(value, 0, 1) as id', 'substring(value, 3, length(value)) as type')
i94visa.show(i94visa.count(), False)
i94visa.count()


In [ ]:
i94visa.write.mode("overwrite").parquet(os.path.join(output_data, 'i94visa'))

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### The Data Dictionary is as follows:

**immigration:-**
* cicid: int8, CICID is a unique number for the immigrants. (No null values found)\n
* i94yr: int4, 4 digit year
* i94mon: int4, Numeric month
* i94cit: int4, country of citizenship
* i94res: int4, country of residence - from where one has travelled. (No null values found)
* i94port: varchar(3), Shows all the valid and invalid codes for Ports
* arrdate: int8, is the Arrival Date in the USA. It is a SAS date numeric field that a permament format has not been applied. (Convert it to timestamp format)
* i94mode: int4, Mode of arrival. There are missing values as well as not reported
* i94addr: varchar(10), Destination address - is where the immigrants resides in USA. Invalid codes are marked as 'other'
* depdate: int8, is the Departure Date from the USA. It is a SAS date numeric field that a permament format has not been applied. (Convert it to timestamp format)
* i94bir: int4, Age of Respondent in Years 
* i94visa: int4, Visa codes 
* count: int4, Used for summary statistics 
* dtadfile: int8, Character Date Field - Date added to I-94 Files - CIC does not use 
* visapost: varchar(5), Department of State where Visa was issued 
* occup: char(3), Occupation that will be performed in U.S. 
* entdepa: char(1), Arrival Flag - admitted or paroled into the U.S.
* entdepd: char(1), Departure Flag - Departed, lost I-94 or is deceased
* entdepu: char(1), Update Flag - Either apprehended, overstayed, adjusted to perm residence
* matflag: char(1), Match flag - Match of arrival and departure records
* biryear: int4, 4 digit year of birth 
* dtaddto: int8, Character Date Field - Date to which admitted to U.S. (allowed to stay until) 
* gender: char(1), Non-immigrant sex 
* insnum: varchar, INS number 
* airline: char(2), Airline used to arrive in U.S. 
* admnum: int8, Admission Number 
* fltno: varchar, Flight number of Airline used to arrive in U.S. 
* visatype: char(2), Class of admission legally admitting the non-immigrant to temporarily stay in U.S.

**Demographics:-**
* City: varchar(50)                   
* State: varchar(50)                   
* Median Age: int4       
* Male Population: int8       
* Female Population: int8
* Total Population: int8 
* Number of Veterans: int8  
* Foreign-born: int8            
* Average Household Size: float  
* State Code: char(2)              
* Race: varchar(100)                  
* Count: int8

**Airport:-**
* identity: varchar(10)       
* type: varchar(50)         
* name: varchar(50)        
* elevation_ft: float 
* continent: varchar(50)    
* iso_country: varchar(50)     
* iso_region: varchar(50)      
* municipality: varchar(50)   
* gps_code: varchar(50)        
* iata_code: varchar(50)       
* local_code: varchar(50)   
* coordinates: varchar  

**Temperature:-**
* dt: timestamp
* AverageTemperature: float
* AverageTemperatureUncertainty: float
* City: varchar(100)
* Country: varchar(100)
* Latitude: varchar(100)
* Longitude: varchar(100)

**i94visa:** [add columns, datatypes and description]

**i94address_state:**

**i94mode:**

**i94port:**

**i94country:**



#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

In [ ]:
# DATEADD(day, cast(arrdate as int),'1960-01-01') as arriv_date,

# I think the problem is that I was using the sparksql date_add instead of the postgresql dateadd